<a href="https://colab.research.google.com/github/Tejesh-45/Myportfolio1/blob/master/Assignment_8_1812.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%writefile library_system.py
from datetime import date, timedelta

class Book:
    def __init__(self, title, author, isbn):
        self.title = title
        self.author = author
        self.isbn = isbn
        self.is_available = True
        self.due_date = None
        self.borrowed_by = None

    def __repr__(self):
        return f"{self.title} by {self.author} (ISBN: {self.isbn})"


class Member:
    def __init__(self, name):
        self.name = name
        self.borrowed_books = []

    def borrow_book(self, book):
        if not book.is_available:
            raise ValueError("Book is already borrowed.")
        book.is_available = False
        book.borrowed_by = self
        book.due_date = date.today() + timedelta(days=14)
        self.borrowed_books.append(book)

    def return_book(self, book):
        if book not in self.borrowed_books:
            raise ValueError("This book was not borrowed by this member.")
        self.borrowed_books.remove(book)
        book.is_available = True
        book.borrowed_by = None
        return book


class Library:
    def __init__(self):
        self.books = []
        self.members = []

    def add_book(self, book):
        self.books.append(book)

    def remove_book(self, isbn):
        for book in self.books:
            if book.isbn == isbn:
                self.books.remove(book)
                return True
        raise ValueError("Book not found.")

    def search_book(self, title):
        return [book for book in self.books if title.lower() in book.title.lower()]

    def register_member(self, member):
        self.members.append(member)

    def issue_book(self, member, book):
        member.borrow_book(book)

    def return_book(self, member, book):
        returned_book = member.return_book(book)
        fine = self.calculate_fine(returned_book)
        return fine

    def calculate_fine(self, book):
        if book.due_date and date.today() > book.due_date:
            days_overdue = (date.today() - book.due_date).days
            return days_overdue * 5
        return 0


Overwriting library_system.py


In [ ]:
%%writefile test_library_system.py
import pytest
from datetime import date, timedelta
from library_system import Book, Member, Library

@pytest.fixture
def setup_library():
    lib = Library()
    book1 = Book("Python 101", "Guido", "ISBN001")
    book2 = Book("AI for All", "Andrew", "ISBN002")
    member = Member("Sai")
    lib.add_book(book1)
    lib.add_book(book2)
    lib.register_member(member)
    return lib, book1, book2, member


def test_add_and_remove_book(setup_library):
    lib, book1, _, _ = setup_library
    assert book1 in lib.books
    lib.remove_book("ISBN001")
    assert book1 not in lib.books


def test_book_cannot_be_borrowed_twice(setup_library):
    lib, book1, _, member = setup_library
    lib.issue_book(member, book1)
    assert not book1.is_available
    with pytest.raises(ValueError):
        lib.issue_book(member, book1)


def test_fine_calculation_for_overdue_books(setup_library):
    lib, book1, _, member = setup_library
    lib.issue_book(member, book1)
    book1.due_date = date.today() - timedelta(days=3)
    fine = lib.return_book(member, book1)
    assert fine == 15  # ₹5 × 3 days


Overwriting test_library_system.py


In [ ]:
!pip install pytest


In [ ]:
!pytest -v


============================= test session starts ==============================
platform linux -- Python 3.12.12, pytest-8.4.2, pluggy-1.6.0 -- /usr/bin/python3
cachedir: .pytest_cache
rootdir: /content
plugins: langsmith-0.4.37, anyio-4.11.0, typeguard-4.4.4
collected 3 items                                                              

test_library_system.py::test_add_and_remove_book PASSED                  [ 33%]
test_library_system.py::test_book_cannot_be_borrowed_twice PASSED        [ 66%]
test_library_system.py::test_fine_calculation_for_overdue_books PASSED   [100%]

============================== 3 passed in 0.02s ===============================


In [ ]:
!pytest -v -k "test_fine_calculation_for_overdue_books"


============================= test session starts ==============================
platform linux -- Python 3.12.12, pytest-8.4.2, pluggy-1.6.0 -- /usr/bin/python3
cachedir: .pytest_cache
rootdir: /content
plugins: langsmith-0.4.37, anyio-4.11.0, typeguard-4.4.4
collected 3 items / 2 deselected / 1 selected                                  

test_library_system.py::test_fine_calculation_for_overdue_books PASSED   [100%]

======================= 1 passed, 2 deselected in 0.01s ========================
